In [1]:

import pandas as pd
import sqlite3
from sqlalchemy.types import Integer, Float, String, DateTime

In [4]:
import pandas as pd
import sqlite3
from pathlib import Path

def csv_to_sqlite(csv_file, db_file, table_name, chunksize=100000):
    # Create database connection
    conn = sqlite3.connect(db_file)
    
    # Read CSV in chunks
    for chunk_number, chunk in enumerate(pd.read_csv(csv_file, chunksize=chunksize)):
        # Write chunk to sqlite
        chunk.to_sql(
            name=table_name,
            con=conn,
            if_exists='append' if chunk_number > 0 else 'replace',
            index=False
        )
        print(f"Processed chunk {chunk_number + 1}")
    
    conn.close()
    print("Completed!")

# Example usage
csv_file = 'results/dec7_combined_no_id.csv'
db_file = 'results.db'
table_name = 'real'

csv_to_sqlite(csv_file, db_file, table_name)


Processed chunk 1
Processed chunk 2
Processed chunk 3
Processed chunk 4
Processed chunk 5
Processed chunk 6
Processed chunk 7
Processed chunk 8
Processed chunk 9
Processed chunk 10
Processed chunk 11
Processed chunk 12
Processed chunk 13
Processed chunk 14
Processed chunk 15
Processed chunk 16
Processed chunk 17
Processed chunk 18
Processed chunk 19
Processed chunk 20
Processed chunk 21
Processed chunk 22
Processed chunk 23
Processed chunk 24
Processed chunk 25
Processed chunk 26
Processed chunk 27
Processed chunk 28
Processed chunk 29
Processed chunk 30
Processed chunk 31
Processed chunk 32
Processed chunk 33
Processed chunk 34
Processed chunk 35
Processed chunk 36
Processed chunk 37
Processed chunk 38
Processed chunk 39
Processed chunk 40
Processed chunk 41
Processed chunk 42
Processed chunk 43
Processed chunk 44
Processed chunk 45
Processed chunk 46
Processed chunk 47
Processed chunk 48
Processed chunk 49
Processed chunk 50
Processed chunk 51
Processed chunk 52
Processed chunk 53
Pr

In [11]:
import duckdb
import glob

def parquets_to_sqlite_duck(parquet_pattern, sqlite_db, table_name):
    # Connect to both databases
    duck_conn = duckdb.connect(':memory:')
    
    # Get list of all parquet files
    parquet_files = glob.glob(parquet_pattern, recursive=True)
    total_files = len(parquet_files)
    
    print(f"Found {total_files} parquet files")
    
    # Create a view of all parquet files
    duck_conn.execute(f"""
        CREATE VIEW parquet_view AS 
        SELECT * FROM read_parquet('{parquet_pattern}')
    """)
    
    # Export to SQLite
    duck_conn.execute(f"""
        INSTALL sqlite;
        LOAD sqlite;
        ATTACH '{sqlite_db}' AS sqlite_db;
    """)
    
    # Copy data to SQLite
    duck_conn.execute(f"""
        INSERT INTO sqlite_db.{table_name} 
        SELECT * FROM parquet_view
    """)
    
    duck_conn.close()
    print("Completed!")

# Usage
synth_path = "results/synth/**/*.parquet"
db_file = 'results.db'
table_name = 'synth'

parquets_to_sqlite_duck(synth_path, db_file, table_name)


Found 5610 parquet files


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Completed!
